In [1]:
import awkward as ak
import pandas as pd
import numpy as np
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents import NanoAODSchema, DelphesSchema
import mplhep as hep
import matplotlib.pyplot as plt
import seaborn as sns
import DM_HEP_AN as dm
from math import pi
hep.style.use("CMS")

In [2]:
ak.__version__

'1.10.4'

In [3]:
#Set the benchmark to analyze
#BM = "bkg"
BM = 'toy'

#Set the number of jets you want to take into account in your analysis
n_jets = 4
n_lep = 4

if BM == 1:
    fname = "/cms/mc/MG5_aMC_v3_1_1/pp2monojet_tjj/Events/run_02/tag_1_delphes_events.root"
    x_sec = 28.85 * 1e-12
elif BM == 2:
    fname = "/cms/mc/MG5_aMC_v3_1_1/pp2monojet_tjj/Events/run_03/tag_1_delphes_events.root"
    x_sec = 0.004521995016 * 1e-12
elif BM == 'toy':
    fname = "/home/tomas/Documents/MG5_aMC_v2_9_15/MonoJet_DM_UdeA/Events/run_19/tag_1_delphes_events.root"
    x_sec = 22.45 * 1e-12
elif BM == "bkg":
    fname = "/cms/mc/MG5_aMC_v3_1_1/Pp2DY_Plus_Jets_Nu/Events/run_05/tag_1_delphes_events.root"
    x_sec = 110.1 * 1e-12

tree_test = dm.Converter(fname)
tree_test.generate(jet_elements = n_jets, e_mu_elements = n_lep)
df = tree_test.df

In [4]:
print(df.shape[0])
print(df.columns)
df.head()

100000
Index(['jet_pt0', 'jet_pt1', 'jet_pt2', 'jet_pt3', 'jet_eta0', 'jet_eta1',
       'jet_eta2', 'jet_eta3', 'jet_phi0', 'jet_phi1', 'jet_phi2', 'jet_phi3',
       'jet_mass0', 'jet_mass1', 'jet_mass2', 'jet_mass3', 'jet_btag0',
       'jet_btag1', 'jet_btag2', 'jet_btag3', 'jet_tautag0', 'jet_tautag1',
       'jet_tautag2', 'jet_tautag3', 'muon_pt0', 'muon_pt1', 'muon_pt2',
       'muon_pt3', 'muon_eta0', 'muon_eta1', 'muon_eta2', 'muon_eta3',
       'muon_phi0', 'muon_phi1', 'muon_phi2', 'muon_phi3', 'muon_charge0',
       'muon_charge1', 'muon_charge2', 'muon_charge3', 'electron_pt0',
       'electron_pt1', 'electron_pt2', 'electron_pt3', 'electron_eta0',
       'electron_eta1', 'electron_eta2', 'electron_eta3', 'electron_phi0',
       'electron_phi1', 'electron_phi2', 'electron_phi3', 'electron_charge0',
       'electron_charge1', 'electron_charge2', 'electron_charge3',
       'missinget_met', 'missinget_phi'],
      dtype='object')


,jet_pt0,jet_pt1,jet_pt2,jet_pt3,jet_eta0,jet_eta1,jet_eta2,jet_eta3,jet_phi0,jet_phi1,...,electron_phi0,electron_phi1,electron_phi2,electron_phi3,electron_charge0,electron_charge1,electron_charge2,electron_charge3,missinget_met,missinget_phi
0,185.678146,155.966736,154.903702,NaN,1.052202,0.269386,1.088477,NaN,1.345680,-2.947849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,323.985870,-0.591095
1,249.687119,99.824074,61.591652,26.744967,0.312900,-0.974402,-1.523153,-1.090339,-1.849687,-0.482363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,229.181107,1.799947
2,62.542458,NaN,NaN,NaN,0.224565,NaN,NaN,NaN,-0.011726,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.375320,3.090276
3,118.184502,50.189079,44.099888,41.835606,-2.583996,-2.141608,-0.997780,-0.362824,-2.353317,-0.405161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.831699,2.023201
4,30.741800,26.874575,NaN,NaN,1.384506,-1.702989,NaN,NaN,-1.429101,2.959735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.556112,0.690696


# **Event Selection**

In [5]:
#Cut p_T^{miss} > 200 GeV
df_cut = df[df['missinget_met'] > 200]

In [6]:
#Cut p_T(j0) > 100 GeV
df_cut = df_cut[df_cut['jet_pt0'] > 100]

In [7]:
#Cut eta(j0) > 2.5 
df_cut = df_cut[np.abs(df_cut['jet_eta0']) < 2.5]

In [8]:
#Cut HT > 110 GeV
#HT := sum(jet_pt) over all the jets per event
#Sumar solo por los que tengan mayor pt que 20
df_cut = df_cut[np.sum(df_cut[[f"jet_pt{i}" for i in range(n_jets)]], axis = 1) > 110]
df_cut

,jet_pt0,jet_pt1,jet_pt2,jet_pt3,jet_eta0,jet_eta1,jet_eta2,jet_eta3,jet_phi0,jet_phi1,...,electron_phi0,electron_phi1,electron_phi2,electron_phi3,electron_charge0,electron_charge1,electron_charge2,electron_charge3,missinget_met,missinget_phi
0,185.678146,155.966736,154.903702,NaN,1.052202,0.269386,1.088477,NaN,1.345680,-2.947849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,323.985870,-0.591095
1,249.687119,99.824074,61.591652,26.744967,0.312900,-0.974402,-1.523153,-1.090339,-1.849687,-0.482363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,229.181107,1.799947
22,251.283127,56.125328,NaN,NaN,1.995831,0.057898,NaN,NaN,-0.907828,-3.028812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.993851,1.996721
30,370.654388,NaN,NaN,NaN,-1.275772,NaN,NaN,NaN,2.086421,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,367.983704,-1.050760
31,212.017166,NaN,NaN,NaN,-1.888465,NaN,NaN,NaN,2.244358,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.292297,-0.828260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,477.165924,26.725201,NaN,NaN,1.758643,-2.674046,NaN,NaN,0.955562,3.051244,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,488.339600,-2.106978
99993,194.852875,NaN,NaN,NaN,-0.525795,NaN,NaN,NaN,1.369322,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.346512,-1.677001
99994,244.013672,113.875984,54.645103,35.188854,-0.638256,0.091436,1.561813,0.580524,1.037827,0.124223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331.516357,-2.092633
99996,226.909790,111.362076,83.172943,NaN,1.981213,1.289445,1.139372,NaN,-1.457395,-2.226123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330.182220,1.661199


In [9]:
df_cut.apply(dm.veto_tag, axis = 1, args = ["b", 4])
#veto_tag(df_cut, "b", n_jets = 4)

0        0
1        0
22       0
30       0
31       0
        ..
99992    0
99993    0
99994    0
99996    0
99998    0
Length: 18064, dtype: int64

In [10]:
df_cut['n_taus'] = df_cut.apply(dm.veto_tag, axis = 1, args = ["tau", n_jets])
df_cut['n_bs'] = df_cut.apply(dm.veto_tag, axis = 1, args = ["b", n_jets])
df_cut['n_ele'] = df_cut.apply(dm.veto_tag, axis = 1, args = ["electron", n_lep])
df_cut['n_mu'] = df_cut.apply(dm.veto_tag, axis = 1, args = ["muon", n_lep])
#df_cut['b_pt'] = df_cut.apply(dm.pt_tag_extractor, args=('b', n_jets), axis = 1)

In [11]:
df_cut = df_cut[df_cut['n_taus'] < 1]
df_cut = df_cut[df_cut['n_bs'] < 1]
df_cut = df_cut[df_cut['n_ele'] < 1]
df_cut = df_cut[df_cut['n_mu'] < 1]

Corregir la función anterior: Contar el número de Taus por evento con las condiciones, si hay al menos uno de ellos, quitar el evento.

In [12]:
df_cut["deltaphi_jet0_met"] = df_cut.apply(dm.DeltaPhi, args = ("jet_phi0", "missinget_phi"), axis = 1)
df_cut["deltaphi_jet1_met"] = df_cut.apply(dm.DeltaPhi, args = ("jet_phi1", "missinget_phi"), axis = 1)
df_cut["deltaphi_jet2_met"] = df_cut.apply(dm.DeltaPhi, args = ("jet_phi2", "missinget_phi"), axis = 1)
df_cut["deltaphi_jet3_met"] = df_cut.apply(dm.DeltaPhi, args = ("jet_phi3", "missinget_phi"), axis = 1)
#df_cut["deltaphi_jet0_jet1"] = df_cut.apply(dm.DeltaPhi, args = ("jet_phi0", "jet_phi1"), axis = 1)

df_copy = df_cut.copy()

In [13]:
df_cut = df_cut[np.abs(df_cut["deltaphi_jet0_met"]) > 0.5]
df_cut = df_cut[(np.abs(df_cut["deltaphi_jet1_met"]) > 0.5) | (df_cut["deltaphi_jet1_met"].isna())]
df_cut = df_cut[(np.abs(df_cut["deltaphi_jet2_met"]) > 0.5) | (df_cut["deltaphi_jet2_met"].isna())]
df_cut = df_cut[(np.abs(df_cut["deltaphi_jet3_met"]) > 0.5) | (df_cut["deltaphi_jet3_met"].isna())]
#df_cut = df_cut[(np.abs(df_cut["deltaphi_jet0_jet1"]) > 3.5) | (df_cut["deltaphi_jet0_jet1"].isna())]
df_cut

,jet_pt0,jet_pt1,jet_pt2,jet_pt3,jet_eta0,jet_eta1,jet_eta2,jet_eta3,jet_phi0,jet_phi1,...,missinget_met,missinget_phi,n_taus,n_bs,n_ele,n_mu,deltaphi_jet0_met,deltaphi_jet1_met,deltaphi_jet2_met,deltaphi_jet3_met
0,185.678146,155.966736,154.903702,NaN,1.052202,0.269386,1.088477,NaN,1.345680,-2.947849,...,323.985870,-0.591095,0,0,0,0,1.936775,-2.356754,-2.697854,NaN
1,249.687119,99.824074,61.591652,26.744967,0.312900,-0.974402,-1.523153,-1.090339,-1.849687,-0.482363,...,229.181107,1.799947,0,0,0,0,2.633552,-2.282310,-0.616516,-1.626951
22,251.283127,56.125328,NaN,NaN,1.995831,0.057898,NaN,NaN,-0.907828,-3.028812,...,233.993851,1.996721,0,0,0,0,-2.904549,1.257653,NaN,NaN
30,370.654388,NaN,NaN,NaN,-1.275772,NaN,NaN,NaN,2.086421,NaN,...,367.983704,-1.050760,0,0,0,0,3.137181,NaN,NaN,NaN
31,212.017166,NaN,NaN,NaN,-1.888465,NaN,NaN,NaN,2.244358,NaN,...,221.292297,-0.828260,0,0,0,0,3.072617,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,477.165924,26.725201,NaN,NaN,1.758643,-2.674046,NaN,NaN,0.955562,3.051244,...,488.339600,-2.106978,0,0,0,0,3.062541,-1.124963,NaN,NaN
99993,194.852875,NaN,NaN,NaN,-0.525795,NaN,NaN,NaN,1.369322,NaN,...,207.346512,-1.677001,0,0,0,0,3.046323,NaN,NaN,NaN
99994,244.013672,113.875984,54.645103,35.188854,-0.638256,0.091436,1.561813,0.580524,1.037827,0.124223,...,331.516357,-2.092633,0,0,0,0,3.130460,2.216856,-1.668888,-2.306888
99996,226.909790,111.362076,83.172943,NaN,1.981213,1.289445,1.139372,NaN,-1.457395,-2.226123,...,330.182220,1.661199,0,0,0,0,-3.118594,2.395863,-1.785649,NaN


In [14]:
x_sec_teo = 7.277487e-5 * 1e-12
if BM == "bkg":
    Luminosity = 12.9
else:
    Luminosity = 12.9
n_mc_ev = df.shape[0]
n_ex_ev = x_sec * Luminosity / (1e-15)
w = n_ex_ev / n_mc_ev
n_ex_ev

289604.99999999994

In [15]:
w

2.8960499999999993

In [16]:
w * df_cut.shape[0]

38836.03049999999

In [17]:
bins = np.concatenate([np.arange(200, 330, 30),
                       np.arange(350, 560, 40),
                       np.arange(590, 800, 50),
                       np.arange(840, 1030, 60),
                       np.arange(1090, 1170, 70),
                       np.array([2000])])
bins

array([ 200,  230,  260,  290,  320,  350,  390,  430,  470,  510,  550,
        590,  640,  690,  740,  790,  840,  900,  960, 1020, 1090, 1160,
       2000])

In [18]:

bins = np.concatenate([np.arange(200, 330, 30),
                       np.arange(350, 560, 40),
                       np.arange(590, 800, 50),
                       np.arange(840, 1030, 60),
                       np.arange(1090, 1170, 70),
                       np.array([2000])])
#bins
counts, ptmiss = np.histogram(df_cut.missinget_met, bins)
counts = counts * w
error = w * counts ** 0.5
di_missing_et = {f"{ptmiss[i]} - {ptmiss[i+1]}" :
                 counts[i] for i in range(len(bins)-1)}

di_missing_et

{'200 - 230': 7639.779899999999,
 '230 - 260': 6481.359899999999,
 '260 - 290': 5027.542799999999,
 '290 - 320': 3912.563549999999,
 '320 - 350': 2887.3618499999993,
 '350 - 390': 3240.6799499999993,
 '390 - 430': 2250.2308499999995,
 '430 - 470': 1807.1351999999995,
 '470 - 510': 1355.3513999999998,
 '510 - 550': 1045.4740499999998,
 '550 - 590': 776.1413999999999,
 '590 - 640': 599.4823499999999,
 '640 - 690': 503.91269999999986,
 '690 - 740': 443.0956499999999,
 '740 - 790': 269.33264999999994,
 '790 - 840': 165.07484999999997,
 '840 - 900': 165.07484999999997,
 '900 - 960': 104.25779999999997,
 '960 - 1020': 55.02494999999999,
 '1020 - 1090': 37.64864999999999,
 '1090 - 1160': 23.168399999999995,
 '1160 - 2000': 46.33679999999999}

In [19]:
error

array([253.13166693, 233.1519889 , 205.34491078, 181.14934079,
       155.61686524, 164.8633524 , 137.37876025, 123.11228111,
       106.61836296,  93.64027585,  80.68196753,  70.90784002,
        65.01053014,  60.96138129,  47.52811145,  37.20887259,
        37.20887259,  29.57061343,  21.4825526 ,  17.76972716,
        13.93972085,  19.71374229])

In [21]:
df_jessica = pd.DataFrame(data = [di_missing_et.keys(), di_missing_et.values(), error]).T
df_jessica.columns = ["bins", "counts", "error"]
df_jessica['bins_lower'] = ptmiss[:-1]
df_jessica
#df_jessica.to_csv(f'/home/tomas/Documents/UdeA_Results/DM/pt_miss_histogram_monojet_BM{BM}_DM_400_GeV.csv')

,bins,counts,error,bins_lower
0,200 - 230,7639.7799,253.131667,200
1,230 - 260,6481.3599,233.151989,230
2,260 - 290,5027.5428,205.344911,260
3,290 - 320,3912.56355,181.149341,290
4,320 - 350,2887.36185,155.616865,320
5,350 - 390,3240.67995,164.863352,350
6,390 - 430,2250.23085,137.37876,390
7,430 - 470,1807.1352,123.112281,430
8,470 - 510,1355.3514,106.618363,470
9,510 - 550,1045.47405,93.640276,510


In [ ]:
df.head()

In [ ]:
if BM == 1:
    bm1_counts = counts * w
if BM == 2:
    bm2_counts = h1 * w
if BM == 'toy':
    bmt_counts = counts

In [ ]:
labs_sizes = 18
f, axs = plt.subplots( figsize=(9, 7))
hep.histplot(counts,
             bins = ptmiss,
             ax=axs, 
             histtype= 'step' ,
             #yerr= h1 ** 0.5 * w, 
             color = 'blue',
             label = 'B.M. toy'
            )
#hep.histplot(bm2_counts,
#             bins = binss1,
#             ax=axs, 
#             histtype= 'step' ,
             #yerr= h1 ** 0.5 * w, 
#             color = 'red',
#             label = 'B.M. 2'
#            )

axs.set_xlabel("$p_T^{miss}$", fontsize = labs_sizes)
axs.set_ylabel('Events',fontsize = labs_sizes)
axs.set_yscale("log")
axs.legend(fontsize = labs_sizes - 6 , loc = 'best')
axs.xaxis.set_tick_params(labelsize= labs_sizes)
axs.yaxis.set_tick_params(labelsize= labs_sizes)
axs.set_title(r'$12.9 \,fb^{-1}(13 \,TeV)$', loc = 'right', fontsize = labs_sizes + 1)
plt.savefig("ptmiss_monojet");